In [33]:
def convert_coco_json_to_csv(filename, img_folder_path, id_limit):
    import pandas as pd
    import json
    
    # COCO2017/annotations/instances_val2017.json
    s = json.load(open(filename, 'r'))
    out_file = filename[:-5] + '.csv'
    out = open(out_file, 'w')
    out.write('images,x1,y1,x2,y2,label\n')

    cat_id_to_label = {}
    for cat in s['categories']:
        cat_id_to_label[cat['id']] = cat['name']
        
    all_ids = []
    for im in s['images']:
        all_ids.append(im['id'])

    all_ids_ann = []
    for ann in s['annotations']:
        image_id = ann['image_id']
        if image_id > id_limit: break
        file_name = s['images'][image_id]['file_name'].replace('data/','')
        all_ids_ann.append(image_id)
        x1 = ann['bbox'][0]
        x2 = ann['bbox'][0] + ann['bbox'][2]
        y1 = ann['bbox'][1]
        y2 = ann['bbox'][1] + ann['bbox'][3]
        label = cat_id_to_label[ann['category_id']]
        out.write('{},{},{},{},{},{}\n'.format(img_folder_path + file_name, x1, y1, x2, y2, label))

    all_ids = set(all_ids)
    all_ids_ann = set(all_ids_ann)
    no_annotations = list(all_ids - all_ids_ann)
    # Output images without any annotations
    for image_id in no_annotations:
        if image_id <= id_limit:
            out.write('{},{},{},{},{},{}\n'.format(img_folder_path + file_name, -1, -1, -1, -1, -1))
    out.close()

    # Sort file by image id
    s1 = pd.read_csv(out_file)
    s1.sort_values('images', inplace=True)
    s1.to_csv(out_file, index=False)

In [ ]:
convert_coco_json_to_csv('./images_thermal_train/coco.json', 'C:/Users/darjan.crncic/Documents/flir_dataset/images_thermal_train/data/', 200)